In [1]:
import os, sqlite3
import pandas as pd
from datetime import datetime, timezone

# ===== CONFIG =====
DB_PATH = r"C:\Users\chris\boxingproject\boxing_odds_staging.db"
OUTPUT_PATH = r"C:\Users\chris\boxingproject\upcoming_event.html"
BOXREC_EVENT_URL = "https://boxrec.com/en/event/924625"   # your link
POSSIBLE_DATE = "2025-12-31"  # placeholder = unconfirmed
OVERRIDE_FIGHT_DATE = None    # e.g. "2025-08-16" to force a specific show; otherwise auto-picks next date

# ===== LOAD LATEST SNAPSHOT =====
if not os.path.exists(DB_PATH):
    raise SystemExit(f"❌ DB not found: {DB_PATH}")

conn = sqlite3.connect(DB_PATH)
latest_sql = """
SELECT *
FROM odds_history
WHERE (event_id, fighter, bookmaker, observed_at) IN (
    SELECT event_id, fighter, bookmaker, MAX(observed_at)
    FROM odds_history
    GROUP BY event_id, fighter, bookmaker
)
"""
df = pd.read_sql(latest_sql, conn)
conn.close()

if df.empty:
    raise SystemExit("❌ No odds in odds_history. Run your ingest first.")

# ===== CLEAN / PREP =====
df["commence_time"] = pd.to_datetime(df["commence_time"], errors="coerce", utc=True)
df["Fight Date"] = df["commence_time"].dt.strftime("%Y-%m-%d")
df["Fight Time (UTC)"] = df["commence_time"].dt.strftime("%H:%M")
df["Matchup"] = df["home_team"].fillna("") + " vs " + df["away_team"].fillna("")
df["fighter"] = df["fighter"].astype(str).str.strip()

# Keep only head-to-head market and drop draw lines
df = df[(df["market"].astype(str).str.lower() == "h2h") & (df["fighter"].str.lower() != "draw")].copy()

# ===== PICK TARGET DATE =====
# Use override if set, else auto-pick next upcoming date (excluding placeholders)
if OVERRIDE_FIGHT_DATE:
    target_date = OVERRIDE_FIGHT_DATE
else:
    today = datetime.now(timezone.utc).date()
    candidates = (
        df[(df["Fight Date"] != POSSIBLE_DATE)]
        .copy()
    )
    if candidates.empty:
        # fall back to earliest available (could be placeholders)
        target_date = df["Fight Date"].min()
    else:
        # next date >= today else earliest future in data
        dates = sorted(set(pd.to_datetime(candidates["Fight Date"]).dt.date))
        future = [d for d in dates if d >= today]
        target_date = str((future[0] if future else dates[0]))

# ===== FILTER TO TARGET DATE =====
day_df = df[df["Fight Date"] == target_date].copy()
if day_df.empty:
    # Show something rather than blank: pick earliest date available
    target_date = df["Fight Date"].min()
    day_df = df[df["Fight Date"] == target_date].copy()

# ===== BEST PRICE PER FIGHTER (for this date) =====
idx_best = day_df.groupby(["event_id","fighter"])["decimal_odds"].idxmax()
best = day_df.loc[idx_best, ["event_id","Matchup","Fight Date","Fight Time (UTC)","fighter","bookmaker","decimal_odds","home_team","away_team"]].copy()

# Build two rows per fight: one for each fighter
# Ensure consistent ordering: show home and away as two entries
def fighter_rows(group):
    # prefer to show in home vs away order if we can match names
    rows = []
    ht, at = group["home_team"].iloc[0], group["away_team"].iloc[0]
    g = group.set_index("fighter")
    # try home
    if ht in g.index:
        rows.append(g.loc[ht])
    # try away
    if at in g.index:
        rows.append(g.loc[at])
    # if any missing (name mismatch), append remaining fighters
    if len(rows) < len(group):
        seen = set([r.name for r in rows])
        for f, r in g.iterrows():
            if f not in seen:
                rows.append(r)
    return rows[:2]  # only two fighters

cards = []
for (ev, matchup), grp in best.groupby(["event_id","Matchup"]):
    grp = grp.sort_values("decimal_odds", ascending=False)
    rows = fighter_rows(grp)
    if len(rows) < 1:
        continue
    # Safe pulls
    date = grp["Fight Date"].iloc[0]
    time_utc = grp["Fight Time (UTC)"].iloc[0] if pd.notna(grp["Fight Time (UTC)"].iloc[0]) else ""
    f1 = rows[0].name
    b1 = rows[0]["bookmaker"]
    o1 = rows[0]["decimal_odds"]
    f2 = rows[1].name if len(rows) > 1 else ""
    b2 = rows[1]["bookmaker"] if len(rows) > 1 else ""
    o2 = rows[1]["decimal_odds"] if len(rows) > 1 else ""

    cards.append({
        "event_id": ev,
        "matchup": matchup,
        "date": date,
        "time_utc": time_utc,
        "f1": f1, "b1": b1, "o1": o1,
        "f2": f2, "b2": b2, "o2": o2
    })

# ===== HTML RENDER =====
def card_html(c):
    # odds formatting
    fmt1 = f"{c['o1']:.2f}" if pd.notna(c["o1"]) else "-"
    fmt2 = f"{c['o2']:.2f}" if pd.notna(c["o2"]) else "-"
    # small helper to avoid 'None'
    def safe(x): return "" if x is None else str(x)
    return f"""
    <div class="fight-card">
      <div class="fight-top">
        <div class="fight-date">{c['date']}{' · ' + c['time_utc'] if c['time_utc'] else ''} UTC</div>
        <div class="fight-matchup">{safe(c['matchup'])}</div>
      </div>
      <div class="fight-odds">
        <div class="side">
          <div class="name">{safe(c['f1'])}</div>
          <div class="price">Best <b>{fmt1}</b></div>
          <div class="book">{safe(c['b1'])}</div>
        </div>
        <div class="vs">vs</div>
        <div class="side">
          <div class="name">{safe(c['f2'])}</div>
          <div class="price">Best <b>{fmt2}</b></div>
          <div class="book">{safe(c['b2'])}</div>
        </div>
      </div>
    </div>
    """

cards_html = "\n".join(card_html(c) for c in cards) if cards else "<div class='empty'>No fights found for this date.</div>"

html = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8" />
<title>Upcoming Fights — {target_date}</title>
<meta name="viewport" content="width=device-width, initial-scale=1" />
<style>
  :root {{
    --bg: #0f1220;
    --card: #141a33;
    --muted: #9fb0d1;
    --line: #233157;
    --brand: #ffcc00;
    --accent: #00ffd0;
    --text: #e8ecf3;
  }}
  * {{ box-sizing: border-box; }}
  body {{
    margin: 0; background: var(--bg); color: var(--text);
    font-family: Segoe UI, system-ui, -apple-system, Arial, sans-serif;
  }}
  .wrap {{
    max-width: 1200px; margin: 0 auto; padding: 24px;
  }}
  header {{
    display: flex; align-items: center; justify-content: space-between; gap: 12px; margin-bottom: 16px;
  }}
  .title {{
    font-size: 22px; font-weight: 800; color: var(--brand);
  }}
  .sub {{
    color: var(--muted); font-size: 14px;
  }}
  .actions a {{
    display: inline-block; padding: 10px 14px; border-radius: 10px; text-decoration: none;
    background: #1f2a52; color: #cdd9ff; border: 1px solid var(--line);
  }}
  .grid {{
    display: grid; grid-template-columns: repeat(auto-fill, minmax(310px,1fr)); gap: 16px;
  }}
  .fight-card {{
    background: var(--card); border: 1px solid var(--line); border-radius: 14px;
    box-shadow: 0 12px 28px rgba(0,0,0,.25); padding: 16px;
    display: flex; flex-direction: column; gap: 12px;
  }}
  .fight-top {{
    display: grid; gap: 6px;
  }}
  .fight-date {{
    color: var(--muted); font-size: 13px;
  }}
  .fight-matchup {{
    font-size: 18px; font-weight: 700; color: #cdd9ff;
  }}
  .fight-odds {{
    display: grid; grid-template-columns: 1fr auto 1fr; align-items: center; gap: 12px;
  }}
  .side .name {{ font-weight: 700; margin-bottom: 4px; }}
  .side .price {{ color: var(--accent); margin-bottom: 2px; }}
  .side .book {{ color: var(--muted); font-size: 13px; }}
  .vs {{
    font-weight: 800; color: var(--muted);
  }}
  .empty {{
    color: var(--muted); text-align: center; padding: 32px 0;
  }}
  footer {{
    margin-top: 18px; display: flex; justify-content: space-between; align-items: center; gap: 12px;
    color: var(--muted); font-size: 13px;
  }}
</style>
</head>
<body>
  <div class="wrap">
    <header>
      <div>
        <div class="title">Upcoming Fights — {target_date}</div>
        <div class="sub">Live best prices pulled from your database. (Times shown in UTC)</div>
      </div>
      <div class="actions">
        <a href="{BOXREC_EVENT_URL}" target="_blank" rel="noopener">View on BoxRec</a>
      </div>
    </header>

    <div class="grid">
      {cards_html}
    </div>

    <footer>
      <div>Generated: {datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")} UTC</div>
      <div>Data source: odds_history → latest snapshot per book</div>
    </footer>
  </div>
</body>
</html>
"""

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ Wrote {OUTPUT_PATH}. Open it in your browser.")


✅ Wrote C:\Users\chris\boxingproject\upcoming_event.html. Open it in your browser.
